# Data PROCESSING And TRANSFORMATION Using PYSPARK (cars csv)

In [2]:
#Start SPARK session

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiDemo") \
    .getOrCreate()

## Importing CARS File

In [3]:
df = spark.read.format('csv').option('header','true').load(r'C:\Users\jsevillz\OneDrive - NTT DATA EMEAL\Desktop\Proyectos y demas\spark\cars.csv')
df.show(n=20)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|   usa|    ford galaxie 500|
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|   usa|    chevrolet impala|


## INITIAL EXPLORATION

In [8]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------+--------------------+
|summary|               mpg|         cylinders|      displacement|        horsepower|           weight|      acceleration|        model_year|origin|                name|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------+--------------------+
|  count|               398|               398|               398|               392|              398|               398|               398|   398|                 398|
|   mean|23.514572864321615| 5.454773869346734|193.42587939698493|104.46938775510205|2970.424623115578|15.568090452261291| 76.01005025125629|  NULL|                NULL|
| stddev| 7.815984312565783|1.7010042445332123|104.26983817119587| 38.49115993282846|846.8417741973268| 2.757688929812676|3.6976266467325862|  NULL|  

In [9]:
df.printSchema()

root
 |-- mpg: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- displacement: string (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- acceleration: string (nullable = true)
 |-- model_year: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- name: string (nullable = true)



In [11]:
print(f'Numero de filas del df: {df.count()}')

Numero de filas del df: 398


In [33]:
features_with_na = [feature for feature in df.columns if df.filter(df[feature].isNull()).count() > 0]

for feature in features_with_na:
    null_percentage = (df.filter(df[feature].isNull()).count() / df.count()) * 100
    print(f'The feature {feature} has {null_percentage:.2f}% of NaN values')


    

The feature horsepower has 1.51% of NaN values


### Ejercicios de Transformación de Datos con PySpark

1. **Filtrar datos por eficiencia de combustible:**
   - Filtra el DataFrame para mostrar solo los coches con una eficiencia de combustible superior a cierto umbral (por ejemplo, mostrar solo los coches con mpg mayor a 30).

2. **Calcular la potencia por peso:**
   - Crea una nueva columna que represente la potencia por peso para cada coche. Para ello, divide la columna de potencia entre la columna de peso.

3. **Convertir la columna de peso a kg:**
   - Convierte la columna de peso de libras a kilogramos.

4. **Calcular el promedio de la eficiencia de combustible por año del modelo:**
   - Agrupa los datos por año del modelo y calcula el promedio de la eficiencia de combustible para cada año.

5. **Establecer valores predeterminados para valores faltantes:**
   - Si hay valores faltantes en la columna de caballos de fuerza ('horsepower'), rellénalos con el valor promedio de la columna.

6. **Agregar una nueva columna de fabricante:**
   - Extrae el fabricante del nombre del modelo y crea una nueva columna con esta información.

7. **Establecer una nueva columna de clase de eficiencia de combustible:**
   - Crea una nueva columna que clasifique los coches en diferentes clases de eficiencia de combustible (por ejemplo, bajo, medio, alto) en función de su mpg.

Estos son solo algunos ejemplos. Puedes combinar y personalizar estos ejercicios según tus necesidades y nivel de habilidad en PySpark. ¡Espero que te diviertas practicando tu análisis y transformación de datos! Si necesitas ayuda con alguno de estos ejercicios específicos, no dudes en preguntar.


In [37]:
# 1. Filtrar datos por eficiencia de combustible

df.select('mpg','name').filter('mpg>30').show()

+----+--------------------+
| mpg|                name|
+----+--------------------+
|31.0| toyota corolla 1200|
|35.0|         datsun 1200|
|31.0|         datsun b210|
|32.0| toyota corolla 1200|
|31.0|       toyota corona|
|32.0|          datsun 710|
|31.0|           fiat x1.9|
|33.0|    honda civic cvcc|
|33.0|         honda civic|
|32.0|        datsun b-210|
|31.5|   honda accord cvcc|
|36.0|       renault 5 gtl|
|33.5|datsun f-10 hatch...|
|33.5|      dodge colt m/m|
|43.1|vw rabbit custom ...|
|36.1|         ford fiesta|
|32.8|    mazda glc deluxe|
|39.4|      datsun b210 gx|
|36.1|    honda civic cvcc|
|31.5|         vw scirocco|
+----+--------------------+
only showing top 20 rows



In [42]:
#2. Calcular la potencia por peso
df2 = df.withColumn('PotenciaPorPeso', round(col('horsepower')/col('weight'),2)).show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+---------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|PotenciaPorPeso|
+----+---------+------------+----------+------+------------+----------+------+--------------------+---------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...|           0.04|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|           0.04|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|           0.04|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|           0.04|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|           0.04|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70| 

In [47]:
# 3. Convertir la columan de peso a kilogramos
df.withColumn('Weight_Kg',round(col('Weight')*0.45,2)).show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+---------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|Weight_Kg|
+----+---------+------------+----------+------+------------+----------+------+--------------------+---------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...|   1576.8|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|  1661.85|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|   1546.2|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|  1544.85|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|  1552.05|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|   usa|    ford galaxie 500|  1953.45|
|14.0|    

In [61]:
#4. Calcular el promedio de eficiencia de combustible por año

df.groupby('model_year').agg(round(mean('mpg'),2).alias('avg_mpg')).show()


+----------+-------+
|model_year|avg_mpg|
+----------+-------+
|        73|   17.1|
|        71|  21.25|
|        70|  17.69|
|        75|  20.27|
|        78|  24.06|
|        77|  23.38|
|        82|  31.71|
|        81|  30.33|
|        79|  25.09|
|        72|  18.71|
|        74|   22.7|
|        76|  21.57|
|        80|   33.7|
+----------+-------+



In [65]:
# 5. Rellenar los valores faltantes con el valor promedio

# Calcular la media de la columna 'horsepower'
horsepower_mean = df.select(mean('horsepower')).collect()[0][0]

df_filled = df.fillna(horsepower_mean,subset=['horsepower'])

In [67]:
#6.Agregar una nueva columna de fabricante
df_filled.withColumn('Fabricante', split('name',' ')[0]).show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|Fabricante|
+----+---------+------------+----------+------+------------+----------+------+--------------------+----------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...| chevrolet|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|     buick|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|  plymouth|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|       amc|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|      ford|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|   usa|    ford galaxie 500|      ford|
|

In [72]:
#7. Crear una nueva columna con valores segun su combustible

df_with_fuel_category = df_filled.withColumn('mpg_category', 
                                                when(df_filled['mpg'] < 30, 'Low') \
                                              .when((df_filled['mpg'] >= 30) & (df_filled['mpg'] <= 50), 'Medium') \
                                              .otherwise('High'))

df_with_fuel_category.select('mpg', 'mpg_category').show()


+----+------------+
| mpg|mpg_category|
+----+------------+
|18.0|         Low|
|15.0|         Low|
|18.0|         Low|
|16.0|         Low|
|17.0|         Low|
|15.0|         Low|
|14.0|         Low|
|14.0|         Low|
|14.0|         Low|
|15.0|         Low|
|15.0|         Low|
|14.0|         Low|
|15.0|         Low|
|14.0|         Low|
|24.0|         Low|
|22.0|         Low|
|18.0|         Low|
|21.0|         Low|
|27.0|         Low|
|26.0|         Low|
+----+------------+
only showing top 20 rows



In [73]:
df_with_fuel_category = df_filled.selectExpr('mpg',
                                             'CASE WHEN mpg < 30 THEN "Low" ' \
                                             'WHEN mpg BETWEEN 30 AND 50 THEN "Medium" ' \
                                             'ELSE "High" END AS mpg_category')

df_with_fuel_category.show()


+----+------------+
| mpg|mpg_category|
+----+------------+
|18.0|         Low|
|15.0|         Low|
|18.0|         Low|
|16.0|         Low|
|17.0|         Low|
|15.0|         Low|
|14.0|         Low|
|14.0|         Low|
|14.0|         Low|
|15.0|         Low|
|15.0|         Low|
|14.0|         Low|
|15.0|         Low|
|14.0|         Low|
|24.0|         Low|
|22.0|         Low|
|18.0|         Low|
|21.0|         Low|
|27.0|         Low|
|26.0|         Low|
+----+------------+
only showing top 20 rows

